In [11]:
import pandas as pd
import numpy as np
import os
import re

In [12]:
def extract_lhg(filename: str):

    pattern = r"L(\d+)H(\d+)G(\d+)"
    match = re.search(pattern, filename)

    if not match:
        return None
    
    L, H, G = map(int, match.groups())
    return {"L": L, "H": H, "G": G}

In [13]:
def _compute_spike_mask_phase_space(u: np.ndarray) -> np.ndarray:
    """
    Dado u (señal centrada: sin media), aplica el método de Phase-Space Thresholding
    (pasos 1–5) y devuelve spike_mask (True donde hay spike).

    NO modifica la señal. Solo detecta.
    """
    n = u.size

    # ---------- Paso 1: derivadas Du y D2u ----------
    Du = np.empty_like(u)
    D2u = np.empty_like(u)

    Du[1:-1] = (u[2:] - u[:-2]) / 2.0
    Du[0] = Du[1]
    Du[-1] = Du[-2]

    D2u[1:-1] = (Du[2:] - Du[:-2]) / 2.0
    D2u[0] = D2u[1]
    D2u[-1] = D2u[-2]

    # ---------- Paso 2: desviaciones estándar + Universal Threshold ----------
    su = np.nanstd(u, ddof=0)
    sDu = np.nanstd(Du, ddof=0)
    sD2u = np.nanstd(D2u, ddof=0)

    if su == 0 and sDu == 0 and sD2u == 0:
        return np.zeros_like(u, dtype=bool)

    lambda_U = np.sqrt(2.0 * np.log(n))

    # ---------- Paso 3: ángulo de rotación θ ----------
    sum_u_D2u = np.nansum(u * D2u)
    sum_u2 = np.nansum(u * u)

    if sum_u2 == 0.0:
        theta = 0.0
    else:
        theta = np.arctan2(sum_u_D2u, sum_u2)

    cos_t = np.cos(theta)
    sin_t = np.sin(theta)

    # ---------- Paso 4: parámetros de las elipses ----------
    # 4.A) (u, Du)
    a_u_Du = lambda_U * su
    b_u_Du = lambda_U * sDu

    # 4.B) (Du, D2u)
    a_Du_D2u = lambda_U * sDu
    b_Du_D2u = lambda_U * sD2u

    # 4.C) (u, D2u) rotada
    A = (lambda_U * su) ** 2
    B = (lambda_U * sD2u) ** 2

    cos2 = cos_t * cos_t
    sin2 = sin_t * sin_t
    denom = cos2 * cos2 - sin2 * sin2

    if np.isclose(denom, 0.0):
        a_u_D2u = np.sqrt(max(A, 0.0))
        b_u_D2u = np.sqrt(max(B, 0.0))
    else:
        a2 = (A * cos2 - B * sin2) / denom
        b2 = (B * cos2 - A * sin2) / denom
        a2 = max(a2, 0.0)
        b2 = max(b2, 0.0)
        a_u_D2u = np.sqrt(a2)
        b_u_D2u = np.sqrt(b2)

    # ---------- Paso 5: detección de spikes ----------
    inside_all = np.ones_like(u, dtype=bool)

    # (u, Du)
    if a_u_Du > 0 and b_u_Du > 0:
        x = u
        y = Du
        inside = (x / a_u_Du) ** 2 + (y / b_u_Du) ** 2 <= 1.0
        inside_all &= inside

    # (Du, D2u)
    if a_Du_D2u > 0 and b_Du_D2u > 0:
        x = Du
        y = D2u
        inside = (x / a_Du_D2u) ** 2 + (y / b_Du_D2u) ** 2 <= 1.0
        inside_all &= inside

    # (u, D2u) rotada
    if a_u_D2u > 0 and b_u_D2u > 0:
        x_prime = u * cos_t + D2u * sin_t
        y_prime = -u * sin_t + D2u * cos_t
        inside = (x_prime / a_u_D2u) ** 2 + (y_prime / b_u_D2u) ** 2 <= 1.0
        inside_all &= inside

    spike_mask = ~inside_all
    return spike_mask


def _replace_spikes_cubic(u: np.ndarray, spike_mask: np.ndarray) -> np.ndarray:
    """
    Reemplaza los spikes (True en spike_mask) en la señal centrada u
    usando polinomio cúbico a través de hasta 12 puntos buenos a cada lado.
    Devuelve una nueva versión de u (u_clean).
    """
    u_clean = u.copy()
    n = u.size
    idx = np.arange(n)

    spike_indices = np.where(spike_mask)[0]
    if spike_indices.size == 0:
        return u_clean

    # Agrupar spikes contiguos en eventos [s, e]
    events = []
    start = spike_indices[0]
    prev = spike_indices[0]

    for k in spike_indices[1:]:
        if k == prev + 1:
            prev = k
        else:
            events.append((start, prev))
            start = prev = k
    events.append((start, prev))

    for (s, e) in events:
        left_good = idx[(idx < s) & (~spike_mask)]
        right_good = idx[(idx > e) & (~spike_mask)]

        left_sel = left_good[-12:]
        right_sel = right_good[:12]

        fit_idx = np.concatenate([left_sel, right_sel])

        if fit_idx.size < 4:
            # Fallbacks simples si no hay puntos suficientes para un cúbico
            spike_range = np.arange(s, e + 1)

            if left_sel.size > 0 and right_sel.size > 0:
                x0 = left_sel[-1]
                x1 = right_sel[0]
                y0 = u_clean[x0]
                y1 = u_clean[x1]
                u_clean[spike_range] = np.interp(spike_range, [x0, x1], [y0, y1])
            elif left_sel.size > 0:
                u_clean[spike_range] = u_clean[left_sel[-1]]
            elif right_sel.size > 0:
                u_clean[spike_range] = u_clean[right_sel[0]]
            continue

        x_fit = fit_idx.astype(float)
        y_fit = u_clean[fit_idx]

        degree = 3
        if fit_idx.size <= 3:
            degree = fit_idx.size - 1

        coeffs = np.polyfit(x_fit, y_fit, degree)

        spike_range = np.arange(s, e + 1, dtype=float)
        u_clean[s:e + 1] = np.polyval(coeffs, spike_range)

    return u_clean


def despike_signal(signal: np.ndarray, max_iter: int = 5):
    """
    Versión ITERATIVA del método Phase-Space Thresholding + reemplazo cúbico.

    Devuelve:
      - clean_signal: señal despikeada (con media repuesta)
      - spike_mask_total: máscara booleana OR de todas las iteraciones
      - n_iter: número de iteraciones efectivamente realizadas (>= 0)
    """
    signal = np.asarray(signal, dtype=float)
    mean_u = np.nanmean(signal)
    u_clean = signal - mean_u  # trabajamos centrados

    n = u_clean.size
    if n < 5:
        return signal.copy(), np.zeros_like(signal, dtype=bool), 0

    spike_mask_total = np.zeros_like(u_clean, dtype=bool)
    prev_mask = np.zeros_like(u_clean, dtype=bool)
    n_iter = 0

    for it in range(max_iter):
        # 1) detectar spikes en la señal ACTUAL u_clean
        spike_mask = _compute_spike_mask_phase_space(u_clean)
        spikes_now = spike_mask.sum()

        # contador de iteraciones en las que SÍ llegamos a computar spike_mask
        n_iter = it + 1

        # Criterios de parada:
        if spikes_now == 0:
            # no hay spikes nuevos en esta iteración → paramos
            break

        if it > 0 and np.array_equal(spike_mask, prev_mask):
            # la máscara no cambió respecto a la iteración anterior → convergió
            break

        # Acumulamos todos los spikes que alguna vez fueron detectados
        spike_mask_total |= spike_mask
        prev_mask = spike_mask.copy()

        # 2) reemplazar spikes en u_clean
        u_clean = _replace_spikes_cubic(u_clean, spike_mask)

    # Al final, reponemos la media
    clean_signal = u_clean + mean_u

    return clean_signal, spike_mask_total, n_iter


In [14]:
VELOCITY_COLUMNS = [
    "V1/X/E(cm/s)",   # componente u
    "V2/Y/N(cm/s)",   # componente v
    "V3/Z/U(cm/s)",   # componente w
]

def process_csv_file(filename: str, output_suffix: str = "_clean"):

    print(f"Procesando archivo: {filename}")
    
    data_info = extract_lhg(filename)
    print("Info LHG:", data_info)

    file_path = './raw_data/' + filename
    df = pd.read_csv(file_path, sep="\t")
    
    for col in VELOCITY_COLUMNS:

        if col not in df.columns:
            print(f"⚠️  Columna {col} no encontrada en {filename}, se omite.")
            continue
        
        print(f"  → Despiking en columna: {col}")

        signal = df[col].values
        
        clean_signal, spike_mask, n_iter = despike_signal(signal, max_iter=5)

        print(f"     Iteraciones efectivas: {n_iter}")
        print(f"     Spikes detectados total: {spike_mask.sum()}")

        clean_col = col + "_clean"
        mask_col = col + "_is_spike"
        
        df[clean_col] = clean_signal
        df[mask_col] = spike_mask
    
    base, ext = os.path.splitext(filename)
    output_name = f"./clean_data/{base}{output_suffix}{ext}"
    
    df.to_csv(output_name, index=False, sep="\t")
    print(f"  ✅ Guardado como: {output_name}\n")


In [15]:
csv_files = [file for file in os.listdir(os.getcwd() + '/raw_data') if file.endswith(".csv")]

print(f"Se encontraron {len(csv_files)} archivos CSV.\n")

for i, filename in enumerate(csv_files, start=1):
    print(f"[{i}/{len(csv_files)}]")
    process_csv_file(filename)

Se encontraron 30 archivos CSV.

[1/30]
Procesando archivo: L220H12G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 12, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
     Iteraciones efectivas: 4
     Spikes detectados total: 58
  → Despiking en columna: V2/Y/N(cm/s)
     Iteraciones efectivas: 5
     Spikes detectados total: 95
  → Despiking en columna: V3/Z/U(cm/s)
     Iteraciones efectivas: 5
     Spikes detectados total: 41
  ✅ Guardado como: ./clean_data/L220H12G16_Probe_1_(A837F)._clean.csv

[2/30]
Procesando archivo: L220H15G16_Probe_1_(A837F)..csv
Info LHG: {'L': 220, 'H': 15, 'G': 16}
  → Despiking en columna: V1/X/E(cm/s)
     Iteraciones efectivas: 4
     Spikes detectados total: 19
  → Despiking en columna: V2/Y/N(cm/s)
     Iteraciones efectivas: 5
     Spikes detectados total: 39
  → Despiking en columna: V3/Z/U(cm/s)
     Iteraciones efectivas: 4
     Spikes detectados total: 36
  ✅ Guardado como: ./clean_data/L220H15G16_Probe_1_(A837F)._clean.csv

[3/30]
Procesando